<a href="https://colab.research.google.com/github/maddogmikeb/Jira/blob/master/TimeInStatus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
from IPython.core.display import clear_output

!pip install -q atlassian-python-api
!pip install -q tqdm

clear_output()

In [2]:
# Configure display
from IPython.core.display import clear_output
from google.colab import data_table
data_table.enable_dataframe_formatter()

def printjson(obj):
  print(json.dumps(obj, indent=2))

clear_output()

In [3]:
# Log in

from IPython.core.display import display, HTML, clear_output
from atlassian import Jira
from google.colab import userdata

jira = Jira(
  url=userdata.get('atlassian_host'),
  username=userdata.get('atlassian_username'),
  password=userdata.get('atlassian_apikey'),
  cloud=True
)

me = jira.myself()

display(HTML('<table><tr><td>' + me["displayName"] + '</td><td><img src="' + me["avatarUrls"]["32x32"] + '"/><td></tr></table>'))


Mike Burns


In [4]:
# Get all issues from jql
# Code to help debug -> https://github.com/atlassian-api/atlassian-python-api/blob/master/atlassian/jira.py

from IPython.display import clear_output, display
from atlassian import Jira

# JQL = 'project = FDSEWMSR AND issuetype not in subTaskIssueTypes() AND "Team[Team]" = d3706851-4fae-4b34-9a25-d4e10c5a45e4 and statuscategory = "done" ORDER BY Rank ASC'
JQL = 'project in ("Short Stay", "TRACE Program") and (((created >= -185d or updated >= -185d or lastViewed >= -185d) and issuetype not in subTaskIssueTypes()) or issuetype in (epic, pattern)) order by created DESC'

limit = None

params = {}
if limit is not None:
  params["maxResults"] = int(limit)
params["fields"] = "key,created,resolutiondate,status,issuetype,project,parent,components,fixVersions,customfield_10168,customfield_10001"
params["jql"] = JQL
params["expand"] = "names"
params["maxResults"] = 100
url = jira.resource_url("search")
start = 0
results = []
results.clear() # really make sure its clear!

names = {}

while True:
  clear_output(wait=True)

  params["startAt"] = int(start)
  response = jira.get(url, params=params)
  if not response:
    break

  if "names" in response:
    names = response["names"]
  issues = response["issues"]
  results.extend(issues)
  total = int(response["total"])
  display("DBG: response: total={total} start={startAt} max={maxResults}".format(**response))
  # If we don't have a limit, and there's more to fetch, keep looping
  if limit is not None or total <= len(response["issues"]) + start:
    break
  start += len(issues)

clear_output()

In [7]:
# Get all the change logs and iterate through them to find all the status changes

from IPython.display import clear_output, display
from atlassian import Jira
import datetime
import pandas as pd
import numpy as np
import copy
from tqdm.contrib.concurrent import process_map
import json

issues = copy.deepcopy(results)

def processIssue(issue):
  try:
    # fix up names
    for key, value in names.items():
      if key in issue["fields"]:
        issue["fields"][value] = issue["fields"].pop(key)

    changelog = jira.get_issue_changelog(issue["key"])
    changes = []
    lastChange = issue["fields"]["Created"]
    for log in changelog["values"]:
      for logitem in log["items"]:
        if logitem["field"].upper() == "STATUS":
          logitem["start"] = lastChange
          logitem["end"] = log["created"]
          lastChange = log["created"]
          changes += [{
                'statusid': logitem["from"],
                'status': logitem["fromString"],
                #'start': logitem["start"],
                #'end': logitem["end"],
                'total': (datetime.datetime.fromisoformat(logitem["end"]) - datetime.datetime.fromisoformat(logitem["start"])).total_seconds()
              }]
          #display(logitem)
    if len(changes) > 0:
      changes += [{
          'statusid': issue["fields"]["Status"]["id"],
          'status': issue["fields"]["Status"]["name"],
          #'start': lastChange,
          #'end': None,
          'total': float('inf')
        }]

      df = pd.DataFrame(changes)
      df.groupby('statusid', as_index=False)['total'].sum()
      df = df.reset_index()
      df = df.replace({None: np.nan})
      for index, row in df.iterrows():
        issue[row["statusid"] + "|" + row["status"]] = row["total"]

    issue["parent"] = None
    issue["grandparent"] = None
    if "Parent" in issue["fields"] and issue["fields"]["Parent"] is not None and "key" in issue["fields"]["Parent"]:
        parent_key = issue["fields"]["Parent"]["key"]
        # parent_summary = issue["fields"]["Parent"]["fields"].get("summary", "")
        issue["parent"] = parent_key # f"[{parent_key}] {parent_summary}"

        parent_response = jira.issue(parent_key, "parent")

        if parent_response and "fields" in parent_response and "parent" in parent_response["fields"]:
            grandparent_key = parent_response["fields"]["parent"]["key"]
            #grandparent_summary = parent_response["fields"]["parent"]["fields"].get("summary", "")
            issue["grandparent"] = grandparent_key # f"[{grandparent_key}] {grandparent_summary}"

    # clean up fields
    issue["created"] = issue["fields"]["Created"]
    issue["resolved"] = issue["fields"]["Resolved"]
    issue["project"] = issue["fields"]["Project"]["name"]
    if "Team" in issue["fields"] and issue["fields"]["Team"] is not None and "name" in issue["fields"]["Team"]:
      issue["team"] = issue["fields"]["Team"]["name"]
    else:
      issue["team"] = None  # or any default value you prefer
    if issue["fields"]["Stream Responsible"] is not None:
      issue["stream_responsible"] = issue["fields"]["Stream Responsible"]["value"]
    else:
      issue["stream_responsible"] = None
    issue["components"] = ",".join([component["name"] for component in issue["fields"]["Components"]])
    issue["fixversions"] = ",".join([component["name"] for component in issue["fields"]["Fix versions"]])
    issue["issuetype"] = issue["fields"]["Issue Type"]["name"]
    issue["url"] = jira.url + "browse/" + issue["key"]

    del issue["fields"]
    del issue["expand"]
    del issue["self"]
    return issue
  except Exception as ex:
    print ("---------------------------------------------------------------------------")
    print ("ERROR")
    printjson(issue)
    print ("---------------------------------------------------------------------------")
    raise

processedIssues = process_map(processIssue, issues, max_workers=10, chunksize=1)
clear_output()
#display( pd.DataFrame(processedIssues) )

In [8]:
# Print

from IPython.core.display import display, HTML, clear_output

import json
import pandas as pd
import numpy as np

#print(json.dumps(processedIssues, indent=2))

df = pd.DataFrame(processedIssues)
df = df.reindex(sorted(df.columns, reverse=True), axis=1)
df = df.replace({None: np.nan})
# display(df)

df.to_excel("output.xlsx", index=False)

clear_output()